<a href="https://colab.research.google.com/github/cheskynd/TFURCPP/blob/main/ThanatoFenestra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ThanatoFenestra**

---
**In this Notebook, You will be guided through the program that powers the ThanatoFenestra Altar.** 

**For this notebook to work, A completed circuit that uses a Raspberry pi as the Microcontroller is required. Also, the required libraries must be installed on the Raspberry Pi.**

**If you do not have a circuit, click [here](https://docs.google.com/presentation/d/19ZnFAh5yts9XgJ08IMe_UG5gRqYu8efAYSAV-OgQSvA/edit?usp=sharing) for the circuit tutorial.**

* **Use Jupyter Notebook**


Below are the libraries and imports needed to run the program.
- CV2 is used to manipulate and display images
- os is used to access files in the Operating System
- Random is used to get random photo addresses
- import "MCP3008" from "gpiozero" is used to set which pins to get data from through the MCP3008 digital to analog convertor
- import "get_monitors" from "screeninfo" is used to get information about the monitors connected to the computer and then in this specific program its used to get the width and height of the screen.
- numpy is used move images across the x and y axis in this program.



In [ ]:
import cv2 as cv
import glob
import os
import random
import numpy as np
from gpiozero import MCP3008
from screeninfo import get_monitors

The block below creates a full screen window.

The display's width is saved in the with variable by using get_monitors()[0], index zero indicates that only the information for the monitors is needed.  If more than one monitors is used, you could change the index for example:
```
width = get_monitors()[1].width
```
would get the width for a second monitor connected to the computer.




In [ ]:
cv.namedWindow("window", cv.WND_PROP_FULLSCREEN)
cv.setWindowProperty("window", cv.WND_PROP_FULLSCREEN, cv.WINDOW_FULLSCREEN)
width = get_monitors()[0].width
height = get_monitors()[0].height

Here, we are setting which pins on the MCP3008 are being used.
* The first pin identifed by "MCP3008(0)" is used by the temperature sensor
* The second pin identified by "MCP3008(1)" is used by the first light sensor
* The third pin identified by "MCP3008(2)" is used by the second light sensor


In [ ]:
tempSens = MCP3008(0)
light_sens1 = MCP3008(2)
light_sens2 = MCP3008(1)

The get_photo function below gets a photo address from the computer, saves it in the img variable by using "cv,imread(photo_ad)" to read the photo. It overwrites the img variable with a resized img and then returns the resized img.

In [ ]:
def get_photo(photo_ad):
    """
    Gets photo and resizes and returns the resized photo.
    :param photo_ad: Image address as a string
    :return: Resized Photo
    """
    img = cv.imread(photo_ad)
    img = cv.resize(img, (width, height))
    return img

The translate function below takes in an image and shift its across the screen. The img parameter is the image being translated. The X coordiante tell the function what point to shift to on the X-axis. The Y coordinate tells the function what point to shift to on the Y-axis.

In [ ]:
def translate(img, x=0, y=0):
    """
    This function translates an image by moving it on the X or Y-Axis
    :param img: Image to be translated
    :param x: Value to move image across the X-axis
    :param y: Value to move image across the Y-axis
    :return: The translated image
    """
    transMat = np.float32([[1, 0, x], [0, 1, y]])
    dimensions = (img.shape[1], img.shape[0])
    return cv.warpAffine(img, transMat, dimensions)

This function creates an overlay img. It is used to manipulate the transparency of an image. The img parameter is the img to be manipulated.
the alpha is the transparancy level of the overlay which takes in a value between 0.0 to 1.0. The higher the alpha value the darker the img gets. the overlay_width and overlay_height are to set the dimension of the overlay.

In [ ]:
def overlay(img, alpha, overlay_width, overlay_height):
    """
    This function modifies the transparency of an image
    :param img: Image used to add transparency
    :param alpha: transparency level of the overlay
    :param overlay_width: The width of the overlay
    :param overlay_height:The height of the overlay
    :return: An overlaid image
    """
    overlay = img.copy()
    output = img.copy()
    cv.rectangle(overlay, (0, 0), (overlay_width, overlay_height), (0, 0, 0), -1)
    cv.addWeighted(overlay, alpha, img, 1 - alpha, 0, output)
    return output

This function show the translated image. This was created to reduce code redunducy.

In [ ]:
def show_translated_img():
    """
    This functions shows the translated image
    :return: None
    """
    translated_photo = translate(photo, rightSens_Var, leftSens_Var)
    cv.imshow('window', translated_photo)
    cv.waitKey(50)


Below is the main block where everything is applied.
One important key is the path variable. This must be changed to specify where the images will be referenced from. 

> ***To avoid errors, make sure the path address only contains images.***


---

The alphas dictionary has temperature values to the left and alpha values to the right. These values are used when the flame is blown, as the tempertaure decreses. The program will reference this dictionary to get the alpha value to apply to the overlay.

---
"initial_sens_val1" and "initial_sens_val2" are variables that save the initial or first readings from the light sensors. The purpose of these variables is to get the ambient lighting level in the setting that the altar is placed. 


In [ ]:
if __name__ == '__main__':
    # get_photo things
    path = "/home/pi/Desktop/TFURCPP-main/ThanatoPics"  # This is a folder with the images that are used in this program.
    filenames = glob.glob(os.path.join(path, "*"))

    photo = get_photo(filenames[random.randint(0, len(filenames) - 1)])

    # The alpha values below are used to set the transparency level of the images
    # The Key is the temperature and the Value is the alpha value
    alphas = {70: 0,
              69: .1,
              68: 0.2,
              67: 0.3,
              66: 0.4,
              65: 0.5,
              64: 0.6,
              63: 0.7,
              62: 0.8,
              61: 0.9,
              60: 1.0}
    # These are the initial sensor values
    initial_sens_val1 = light_sens1.value * 1000
    initial_sens_val2 = light_sens2.value * 1000

The block below activates the while loop which continuesly gets the values from our sensors and scales them to be used later.

In [ ]:
    while True:
        # All sensor values are scaled by 1000
        light_sens1_val = light_sens1.value * 1000
        light_sens2_val = light_sens2.value * 1000
        temp_value = (((((tempSens.value * 1000) / 1024) * 5) - 0.5) * 100)

        translated_photo = translate(photo)
        
        print('\n',initial_sens_val1,initial_sens_val2,'\n')
        print(temp_value,light_sens1_val, light_sens2_val)

Here we set a temperature of 70 celcius as the minimum at which images could be displayed.
Images are minipulated in intervals.
* If the light sensors detects a value thats greater than the initail value by a difference of 20 then it will display an image
* if the difference is 130 or greater than the images will shift either up/down or left/right depending on the sensor.
* If the value is 200 or greater than the program will change the image.

In [ ]:
        if temp_value > 70:

            # Display an image if light_sens1_val and light_sens2_val are equal of greater that then start_val
            if light_sens1_val > initial_sens_val1+20 or light_sens1_val > initial_sens_val2+20:
                # Displays window and image.
                cv.imshow('window', translated_photo)
                cv.waitKey(50)

            # flicker the image on the X-axis if light_sens1_val is between the flicker_val and change_img_val
            if light_sens1_val > initial_sens_val1 + 130:
                leftSens_Var = 0
                rightSens_Var = random.randint(-17, 17)
                show_translated_img()

            # flicker the image on the Y-axis if light_sens2_val is between the flicker_val and change_img_val
            if light_sens2_val > initial_sens_val2 + 130:
                rightSens_Var = 0
                leftSens_Var = random.randint(-17, 17)
                show_translated_img()

            # change the image if the light sensor values are above the change_img_val value
            if light_sens1_val > initial_sens_val1 + 200 or light_sens2_val > initial_sens_val2+ 200:
                photo = get_photo(filenames[random.randint(0, len(filenames) - 1)])
                cv.imshow('window', translated_photo)
                cv.waitKey(10)

Here if the temperature is greater than 70 degrees celcius and the candle is blown or the sensor values are within a range of 30 compared to the initial sensor value, than the program iterates through images until the temperature falls below 70 degrees.

In [ ]:
        # This will iterate through the Images once the candle is turned off without any overlay
        if temp_value > 70 and light_sens1_val < initial_sens_val1+30 and light_sens2_val < initial_sens_val2+30:
            photo = get_photo(filenames[random.randint(0, len(filenames) - 1)])
            cv.imshow('window', translated_photo)
            cv.waitKey(10)

This block acts like the block above with the difference being that once the temperature falls to 70 or below but above 60 transparency will kick in.

In [ ]:
        # This will iterate through the Images once the candle is turned off with overlay as long as the temperature
        # is between 55 and 60
        if 60 < temp_value < 70 and initial_sens_val1+30 > light_sens1_val and initial_sens_val2+30 > light_sens2_val:
            int_temp_value = int(temp_value)  # coverts temperature to integer value
            photo = get_photo(filenames[random.randint(0, len(filenames) - 1)])
            overlaid = overlay(translated_photo, alphas[int_temp_value], overlay_width=width, overlay_height=height)
            cv.imshow('window', overlaid)
            cv.waitKey(10)

In the code block below we set a value a 60 as the temperature at which the screen will display a dark overlay img. If the temp_value is higher than 60 then the image will be visible.

In [ ]:
        # If the temperature value is below 60: a black screen will be shown
        if temp_value < 60:
            overlaid = overlay(translated_photo, 1, overlay_width=width, overlay_height=height)
            cv.imshow('window', overlaid)
            cv.waitKey(10)